In [1]:
import json
import pickle
import csv
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import random

import tensorflow as tf
from tensorflow import keras

2023-06-05 16:44:25.878807: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 16:44:28.733491: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Convert to pressure
def convert_to_Pressure(hyai,hybi,PS,P0):
    Dimension1=hyai.shape
    Dimension2=PS.shape
    Pressure = np.zeros([Dimension2[1],Dimension1[0]])
    
    for i in range(Dimension2[1]):
        Pressure[i,:] = (P0*hyai[:] + PS[0,i]*hybi[:])
    return Pressure

In [3]:
ne4_grid_info = xr.open_dataset('./test_data/E3SM-MMF_ne4_grid-info.orig.nc')
lon = ne4_grid_info.lon.values
lat = ne4_grid_info.lat.values
area = ne4_grid_info.area.values
hyai = ne4_grid_info.hyai.values
hybi = ne4_grid_info.hybi.values
hyam = ne4_grid_info.hyam.values
hybm = ne4_grid_info.hybm.values
PS = ne4_grid_info.PS.values
P0 = ne4_grid_info.P0.values

1.000000017058161

In [15]:
ncol = 384
nlev = 60

In [16]:
Prei = convert_to_Pressure(hyai,hybi,PS,P0)
Prem = convert_to_Pressure(hyam,hybm,PS,P0)

In [17]:
dPre = Prei[:,1:nlev+1] - Prei[:,0:nlev] 

In [18]:
import math 
print (area.sum()/(4*math.pi)) 

1.0


In [19]:
mweightpre    =  dPre/9.8  # dP/g

In [20]:
mweightpre = np.zeros([ncol,nlev])
for k in range(nlev):
    mweightpre[:,k]    =  area*dPre[:,k]/9.8/(4*math.pi)

In [9]:
Prem.shape

(384, 60)

In [22]:
import os
import netCDF4 as nc
# Create a new NetCDF file
filename = "mass_weight_area_pressure.nc"
        
ncfile = nc.Dataset(filename, "w", format="NETCDF4")

# Define the dimensions
time_dim = ncfile.createDimension("time", None)  # Unlimited dimension
lat_dim = ncfile.createDimension("ncol", ncol)
pre2_dim = ncfile.createDimension("nlev2", nlev+1)
pre_dim = ncfile.createDimension("nlev", nlev)
# Create variables
time_var = ncfile.createVariable("time", "f8", ("time",))
lon_var = ncfile.createVariable("lon", "f4", ("ncol",))
lat_var = ncfile.createVariable("lat", "f4", ("ncol",))
area_var = ncfile.createVariable("area", "f4", ("ncol",))
data_var3 = ncfile.createVariable("mweightpre", "f8", ("nlev","ncol"))
data_var4 = ncfile.createVariable("Pressurem", "f8", ("nlev","ncol"))
data_var5 = ncfile.createVariable("Pressurei", "f8", ("nlev2","ncol"))

data_var3.long_name = "Area Mass weight coefficient (area*dP/g) normalized vertically and by 4pi"
data_var4.long_name = "Pressure at mid level"
data_var5.long_name = "Pressure at interface"

# Assign values to variables
time_var[:] = [1]  # Example time values
lon_var[:] = lon    # Example latitude values
lat_var[:] = lat  # Example longitude values
area_var[:] = area  # Example longitude values
data_var3[:,:] = np.transpose(mweightpre)
data_var4[:,:] = np.transpose(Prem)
data_var5[:,:] = np.transpose(Prei)
# Add global attributes
ncfile.description = "R2"
ncfile.history = "Created by Liran"

# Close the NetCDF file
ncfile.close()
    